In [ ]:
pip install python-chess


In [ ]:
!pip install pygame


In [ ]:
import chess
import random

class EfficientChessAI:
    def __init__(self):
        self.board = chess.Board()

    def choose_move(self):
        # Select a random legal move
        legal_moves = list(self.board.legal_moves)
        if not legal_moves:
            return None
        return random.choice(legal_moves)

    def make_move(self, move):
        # Apply the move to the board if it's legal
        if move in self.board.legal_moves:
            self.board.push(move)
        else:
            raise ValueError(f"Illegal move attempted: {move}\nCurrent board:\n{self.board}")

    def play_game(self, opponent_ai):
        while not self.board.is_game_over():
            print("Current Board:")
            print(self.board)
            print("Legal moves for AI:", list(self.board.legal_moves))

            # AI's move
            move = self.choose_move()
            if move is None:
                break
            self.make_move(move)
            opponent_ai.make_move(move)  # Synchronize the opponent's board with the AI's move

            print("AI Move:")
            print(self.board)

            # Opponent's move
            print("Opponent's Turn")
            opponent_move = opponent_ai.choose_move()
            if opponent_move is None:
                break

            try:
                opponent_ai.make_move(opponent_move)  # Opponent makes its move
                self.make_move(opponent_move)        # Synchronize AI's board with the opponent's move
            except ValueError as e:
                print(e)
                break

            print("Opponent Move:")
            print(self.board)

        print("Game Over")
        print(self.board.result())

if __name__ == "__main__":
    ai_player = EfficientChessAI()
    opponent = EfficientChessAI()
    ai_player.play_game(opponent_ai=opponent)



In [ ]:
import chess
import random

class EfficientChessAI:
    def __init__(self):
        self.board = chess.Board()

    def choose_move(self):
        # Select a random legal move
        legal_moves = list(self.board.legal_moves)
        if not legal_moves:
            return None
        return random.choice(legal_moves)

    def make_move(self, move):
        # Apply the move to the board
        if move in self.board.legal_moves:
            self.board.push(move)
        else:
            raise ValueError(f"Illegal move attempted: {move}\nCurrent board:\n{self.board}")

    def play_game(self, opponent_ai):
        while not self.board.is_game_over():
            # AI's turn
            print("AI's Turn")
            ai_move = self.choose_move()
            if ai_move is None:
                print("AI has no moves left.")
                break
            self.make_move(ai_move)
            opponent_ai.board.push(ai_move)  # Synchronize opponent's board
            print("AI Move:")
            print(self.board)

            if self.board.is_game_over():
                break

            # Opponent's turn
            print("Opponent's Turn")
            opponent_move = opponent_ai.choose_move()
            if opponent_move is None:
                print("Opponent has no moves left.")
                break
            opponent_ai.make_move(opponent_move)
            self.board.push(opponent_move)  # Synchronize AI's board
            print("Opponent Move:")
            print(self.board)

        print("Game Over")
        print(self.board.result())

if __name__ == "__main__":
    ai_player = EfficientChessAI()
    opponent = EfficientChessAI()
    ai_player.play_game(opponent_ai=opponent)



In [ ]:
import chess
import random
import csv

class EfficientChessAI:
    def __init__(self):
        self.board = chess.Board()
        self.moves_log = []  # To log all moves in UCI format for submission

    def choose_move(self):
        # Select a random legal move
        legal_moves = list(self.board.legal_moves)
        if not legal_moves:
            return None
        return random.choice(legal_moves)

    def make_move(self, move):
        # Apply the move to the board
        if move in self.board.legal_moves:
            self.board.push(move)
            self.moves_log.append(move.uci())  # Log the move in UCI format
        else:
            raise ValueError(f"Illegal move attempted: {move}\nCurrent board:\n{self.board}")

    def play_game(self, opponent_ai):
        while not self.board.is_game_over():
            # AI's turn
            ai_move = self.choose_move()
            if ai_move is None:
                break
            self.make_move(ai_move)
            opponent_ai.board.push(ai_move)  # Synchronize opponent's board

            if self.board.is_game_over():
                break

            # Opponent's turn
            opponent_move = opponent_ai.choose_move()
            if opponent_move is None:
                break
            opponent_ai.make_move(opponent_move)
            self.board.push(opponent_move)  # Synchronize AI's board

        print("Game Over")
        print(self.board.result())

    def save_submission(self, filename="submission.csv"):
        # Save the moves log to a CSV file
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["id", "move"])
            for i, move in enumerate(self.moves_log, 1):
                writer.writerow([i, move])
        print(f"Submission saved to {filename}")

if __name__ == "__main__":
    ai_player = EfficientChessAI()
    opponent = EfficientChessAI()
    ai_player.play_game(opponent_ai=opponent)
    ai_player.save_submission()


In [ ]:
import chess
import random
import csv
import os

class EfficientChessAI:
    def __init__(self):
        self.board = chess.Board()
        self.moves_log = []  # To log moves in UCI format for submission

    def choose_move(self):
        # Select a random legal move
        legal_moves = list(self.board.legal_moves)
        if not legal_moves:
            return None
        return random.choice(legal_moves)

    def make_move(self, move):
        if move in self.board.legal_moves:
            self.board.push(move)
            self.moves_log.append(move.uci())  # Log the move in UCI format
        else:
            raise ValueError(f"Illegal move attempted: {move}")

    def play_game(self, opponent_ai):
        while not self.board.is_game_over():
            # AI's turn
            ai_move = self.choose_move()
            if ai_move is None:
                break
            self.make_move(ai_move)
            opponent_ai.board.push(ai_move)

            if self.board.is_game_over():
                break

            # Opponent's turn
            opponent_move = opponent_ai.choose_move()
            if opponent_move is None:
                break
            opponent_ai.make_move(opponent_move)
            self.board.push(opponent_move)

    def save_submission(self, filename="submission.csv"):
        # Ensure we save to /kaggle/working/
        output_path = os.path.join("/kaggle/working", filename)
        with open(output_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["id", "move"])
            for i, move in enumerate(self.moves_log, 1):
                writer.writerow([i, move])
        print(f"Submission saved to {output_path}")


if __name__ == "__main__":
    ai_player = EfficientChessAI()
    opponent = EfficientChessAI()
    ai_player.play_game(opponent_ai=opponent)
    ai_player.save_submission()


In [ ]:
import chess
import random
import csv
import os

class EfficientChessAI:
    def __init__(self):
        self.board = chess.Board()
        self.moves_log = []

    def choose_move(self):
        # Select a random legal move
        legal_moves = list(self.board.legal_moves)
        if not legal_moves:
            return None
        return random.choice(legal_moves)

    def make_move(self, move):
        # Apply the move to the board if it's legal
        try:
            if move in self.board.legal_moves:
                self.board.push(move)
                self.moves_log.append(move.uci())
            else:
                raise ValueError(f"Illegal move attempted: {move}\nCurrent board:\n{self.board}")
        except ValueError as e:
            print(e)

    def play_game(self, opponent_ai):
        print("Starting game...")
        while not self.board.is_game_over():
            print("Current Board:")
            print(self.board)

            # AI's move
            move = self.choose_move()
            if move is None:
                print("No legal moves available.")
                break
            print(f"AI's move: {move}")
            self.make_move(move)

            # Opponent's move
            opponent_move = opponent_ai.choose_move()
            if opponent_move is None:
                print("Opponent has no legal moves.")
                break
            print(f"Opponent's move: {opponent_move}")
            self.make_move(opponent_move)

        print("Game Over")
        print(f"Result: {self.board.result()}")

    def save_submission(self, output_path="/kaggle/working/submission.csv"):
        print(f"Saving submission to {output_path}...")
        try:
            with open(output_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["id", "move"])
                for i, move in enumerate(self.moves_log, 1):
                    writer.writerow([i, move])
            print(f"Submission saved to {output_path}")
        except Exception as e:
            print(f"Error while saving submission: {e}")

if __name__ == "__main__":
    ai_player = EfficientChessAI()
    opponent = EfficientChessAI()

    try:
        ai_player.play_game(opponent_ai=opponent)
        ai_player.save_submission()

        # Debugging Outputs
        print("\nDebugging Output:")
        print(f"Current directory: {os.getcwd()}")
        print(f"Files in /kaggle/working/: {os.listdir('/kaggle/working/')}")
        
        # Verify the content of the saved file
        submission_path = "/kaggle/working/submission.csv"
        if os.path.exists(submission_path):
            with open(submission_path, 'r') as file:
                print("Submission File Content:")
                print(file.read())
        else:
            print("Error: submission.csv file not found in /kaggle/working/")

    except Exception as main_exception:
        print(f"An error occurred: {main_exception}")


In [ ]:
import chess
import csv
import random


class EfficientChessAI:
    def __init__(self):
        # Initialize the chess board and moves log
        self.board = chess.Board()
        self.moves_log = []

    def choose_move(self):
        """Select a random legal move."""
        legal_moves = list(self.board.legal_moves)
        if not legal_moves:
            return None
        return random.choice(legal_moves)  # Choose a random legal move

    def make_move(self, move):
        """Apply a move to the board if legal."""
        try:
            self.board.push(move)
            self.moves_log.append(move.uci())
        except ValueError:
            raise ValueError(f"Illegal move attempted: {move}\nCurrent board:\n{self.board}")

    def play_game(self, opponent_ai):
        """Simulate a game between two AIs."""
        while not self.board.is_game_over():
            if self.board.can_claim_threefold_repetition():
                print("Game ended in a draw by threefold repetition.")
                break

            # AI makes its move
            ai_move = self.choose_move()
            if ai_move is None:
                break
            self.make_move(ai_move)

            # Opponent makes its move
            opponent_move = opponent_ai.choose_move()
            if opponent_move is None:
                break
            opponent_ai.make_move(opponent_move)

        # Print the result of the game
        print(f"Game result: {self.board.result()}")

    def save_submission(self):
        """Save the game moves log as a CSV file."""
        with open("/kaggle/working/submission.csv", mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["id", "move"])
            for i, move in enumerate(self.moves_log, start=1):
                writer.writerow([i, move])
        print("Submission saved as submission.csv")


if __name__ == "__main__":
    # Initialize the AI players
    ai_player = EfficientChessAI()
    opponent = EfficientChessAI()

    # Play the game
    ai_player.play_game(opponent_ai=opponent)

    # Save the submission file
    ai_player.save_submission()






In [ ]:
import os

if os.path.exists("/kaggle/working/submission.csv"):
    print("File saved successfully.")
else:
    print("File not found.")


In [ ]:
print(f"Moves log: {ai_player.moves_log}")
ai_player.save_submission()
print("Save submission method executed.")


In [ ]:
if __name__ == "__main__":
    # Both AI instances share the same chess board
    shared_board = chess.Board()

    # Initialize the AI players
    ai_player = EfficientChessAI()
    opponent = EfficientChessAI()

    # Play the game
    ai_player.play_game(opponent_ai=opponent)
    
    # Debug: Check moves log
    print(f"Moves log: {ai_player.moves_log}")

    # Save the submission file
    ai_player.save_submission()

    # Debug: Verify file creation
    import os
    if os.path.exists("/kaggle/working/submission.csv"):
        print("File saved successfully.")
    else:
        print("File not found.")


In [ ]:
from Chessnut import Game
import random
import csv
from kaggle_environments import make

# Create a custom chess bot with improved logic
def chess_bot(obs):
    """
    Simple chess bot that prioritizes checkmates, then captures, queen promotions, then randomly moves.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation (e.g., "e2e4")
    """
    # Parse the board state
    game = Game(obs.board)
    moves = list(game.get_moves())

    # 1. Prioritize checkmate moves
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Check for captures
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            return move

    # 3. Prioritize queen promotions
    for move in moves:
        if "q" in move.lower():
            return move

    # 4. If no other options, pick a random move
    return random.choice(moves)


# Simulate a match and save game logs
class ChessGameSimulator:
    def __init__(self):
        self.env = make("chess", debug=True)
        self.moves_log = []

    def play_game(self, player1, player2):
        """
        Run a game between two agents and save the moves log.

        Args:
            player1: The first player's agent function or filename.
            player2: The second player's agent function or filename.
        """
        # Run the game
        result = self.env.run([player1, player2])

        # Save the moves log
        for turn in result:
            if turn[0].observation and "board" in turn[0].observation:
                self.moves_log.append(turn[0].observation.board)

        # Return the game result
        print("Final Game State:")
        for agent in result[-1]:
            print(f"\tStatus: {agent.status}, Reward: {agent.reward}, Time Left: {agent.observation.remainingOverageTime}")

    def save_submission(self, filename="submission.csv"):
        """
        Save the moves log as a CSV file for submission.

        Args:
            filename: The filename for the submission file.
        """
        with open(f"/kaggle/working/{filename}", mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["id", "board"])
            for i, board in enumerate(self.moves_log, start=1):
                writer.writerow([i, board])
        print(f"Submission saved as {filename}")


if __name__ == "__main__":
    # Initialize the simulator
    simulator = ChessGameSimulator()

    # Play a game between the custom chess bot and a random agent
    simulator.play_game(chess_bot, "random")

    # Save the game logs for submission
    simulator.save_submission()


In [ ]:
from Chessnut import Game
import random


def chess_bot(obs):
    """
    Simple chess bot that prioritizes checkmates, captures, queen promotions, and random moves.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation (e.g., "e2e4").
    """
    # Parse the board state using Chessnut
    game = Game(obs.board)
    moves = list(game.get_moves())

    # 1. Checkmate moves
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Capture moves
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            return move

    # 3. Queen promotions
    for move in moves:
        if "q" in move.lower():
            return move

    # 4. Random fallback
    return random.choice(moves)


In [ ]:
from kaggle_environments import make

# Create the chess environment
env = make("chess", debug=True)

# Test the chess bot against a random agent
result = env.run([chess_bot, "random"])  # Pass chess_bot directly

# Display the final game state
for agent in result[-1]:
    print(f"Status: {agent.status}, Reward: {agent.reward}")



In [ ]:
from kaggle_environments import make

# Create the chess environment
env = make("chess", debug=True)

# Test your chess bot against the random agent
result = env.run([chess_bot, "random"])

# Display the final game state
for agent in result[-1]:
    print(f"Status: {agent.status}, Reward: {agent.reward}")


In [ ]:
from kaggle_environments import make

# Create the chess environment
env = make("chess", debug=True)

# Test your chess bot against itself
result = env.run([chess_bot, chess_bot])

# Display the final game state
for agent in result[-1]:
    print(f"Status: {agent.status}, Reward: {agent.reward}")


In [ ]:
def random_bot(obs):
    """
    A bot that chooses a random legal move.
    """
    game = Game(obs.board)
    moves = list(game.get_moves())
    return random.choice(moves)

# Test your chess bot against the random_bot
result = env.run([chess_bot, random_bot])

# Display the final game state
for agent in result[-1]:
    print(f"Status: {agent.status}, Reward: {agent.reward}")


In [ ]:
env.render(mode="ipython", width=1000, height=1000)


In [ ]:
from Chessnut import Game
import random

def chess_bot(obs):
    """
    Simple chess bot that prioritizes checkmates, captures, queen promotions, and random moves.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation (e.g., "e2e4").
    """
    game = Game(obs.board)
    moves = list(game.get_moves())

    # 1. Checkmate moves
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Capture moves
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            return move

    # 3. Queen promotions
    for move in moves:
        if "q" in move.lower():
            return move

    # 4. Random fallback
    return random.choice(moves)


In [ ]:
# Define the bot code as a string
bot_code = """
from Chessnut import Game
import random

def chess_bot(obs):
    \"""
    Simple chess bot that prioritizes checkmates, captures, queen promotions, and random moves.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation (e.g., "e2e4").
    \"""
    game = Game(obs.board)
    moves = list(game.get_moves())

    # 1. Checkmate moves
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Capture moves
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            return move

    # 3. Queen promotions
    for move in moves:
        if "q" in move.lower():
            return move

    # 4. Random fallback
    return random.choice(moves)
"""

# Write the bot code to main.py
with open("/kaggle/working/main.py", "w") as file:
    file.write(bot_code)

print("main.py has been saved to /kaggle/working/")


In [ ]:
# Step 1: Save the bot logic into main.py
bot_code = """
from Chessnut import Game
import random

def chess_bot(obs):
    \"""
    Simple chess bot that prioritizes checkmates, captures, queen promotions, and random moves.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation (e.g., "e2e4").
    \"""
    game = Game(obs.board)
    moves = list(game.get_moves())

    # 1. Checkmate moves
    for move in moves:
        g = Game(obs.board)
        g.apply_move(move)
        if g.status == Game.CHECKMATE:
            return move

    # 2. Capture moves
    for move in moves:
        if game.board.get_piece(Game.xy2i(move[2:4])) != ' ':
            return move

    # 3. Queen promotions
    for move in moves:
        if "q" in move.lower():
            return move

    # 4. Random fallback
    return random.choice(moves)
"""

# Write the bot code to main.py
with open("/kaggle/working/main.py", "w") as file:
    file.write(bot_code)

print("main.py has been saved to /kaggle/working/")

# Step 2: Create a .tar.gz archive for submission
!tar -czvf submission.tar.gz -C /kaggle/working main.py

print("submission.tar.gz has been created.")


1. Move Scoring
Goal: Evaluate moves based on piece values and board position.
You can assign scores to moves based on capturing high-value pieces or achieving positional advantages.

Implementation:

In [ ]:
def score_move(game, move):
    """
    Score a move based on the value of the piece being captured.

    Args:
        game: The current game object.
        move: The move in UCI notation.

    Returns:
        An integer score for the move.
    """
    # Piece values
    piece_value = {"p": 1, "n": 3, "b": 3, "r": 5, "q": 9, "k": 0}
    
    # Determine the target square
    target_square = move[2:4]
    
    # Get the piece on the target square
    piece = game.board.get_piece(Game.xy2i(target_square))
    
    # Return the score based on the captured piece
    return piece_value.get(piece.lower(), 0) if piece != " " else 0

def chess_bot(obs):
    """
    Improved chess bot that uses move scoring.
    """
    game = Game(obs.board)
    moves = list(game.get_moves())
    
    # Score all moves
    scored_moves = [(move, score_move(game, move)) for move in moves]
    
    # Choose the move with the highest score
    best_move = max(scored_moves, key=lambda x: x[1], default=(None, 0))[0]
    
    return best_move if best_move else random.choice(moves)


2. Integrate a Chess Engine
Using a chess engine like Stockfish will significantly improve your bot's decision-making by leveraging its evaluation algorithms.

Setup:
Install Stockfish:

In [ ]:
!pip install chess


In [ ]:
import chess.engine

def chess_bot_with_stockfish(obs):
    """
    A chess bot using Stockfish for move selection.
    """
    # Create a Stockfish engine instance
    with chess.engine.SimpleEngine.popen_uci("/path/to/stockfish") as engine:
        # Create a chess board from the FEN string
        board = chess.Board(obs.board)
        
        # Let Stockfish calculate the best move
        result = engine.play(board, chess.engine.Limit(time=0.1))  # 0.1-second time limit
        return result.move.uci()


3. Analyze Results
Save Game Logs:
Log all moves into a file for later analysis

In [ ]:
def save_game_log(moves, filename="game_log.txt"):
    with open(filename, "w") as file:
        for move in moves:
            file.write(move + "\n")


Review Opening Strategies:
Predefine popular openings to help your bot gain an early advantage.
Use an opening book to guide the first few moves.

In [ ]:
def chess_bot(obs):
    """
    Improved chess bot with opening strategies.
    """
    # Define common openings
    openings = {
        "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1": "e2e4",
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1": "d2d4"
    }

    # If the board matches an opening position, play the predefined move
    if obs.board in openings:
        return openings[obs.board]

    # Default logic for move scoring
    game = Game(obs.board)
    moves = list(game.get_moves())

    # Simple scoring function
    def score_move(game, move):
        piece_value = {"p": 1, "n": 3, "b": 3, "r": 5, "q": 9, "k": 0}
        target_square = move[2:4]
        piece = game.board.get_piece(Game.xy2i(target_square))
        return piece_value.get(piece.lower(), 0) if piece != " " else 0

    # Score all moves and pick the best one
    scored_moves = [(move, score_move(game, move)) for move in moves]
    best_move = max(scored_moves, key=lambda x: x[1], default=(None, 0))[0]

    return best_move if best_move else random.choice(moves)


4. Handle Special Cases
Detect and Avoid Repetition:
Use the can_claim_threefold_repetition() method to detect repetition and break the loop.

In [ ]:
def leads_to_repetition(board, move):
    """
    Simulates the given move to check if it leads to a threefold repetition.
    Args:
        board (chess.Board): The current chess board.
        move (chess.Move): The move to simulate.

    Returns:
        bool: True if the move leads to a threefold repetition, False otherwise.
    """
    board.push(move)
    result = board.can_claim_threefold_repetition()
    board.pop()
    return result

def chess_bot(obs):
    """
    Chess bot combining predefined openings, Stockfish, and heuristics.
    Includes handling for threefold repetition.
    """
    board = chess.Board(obs.board)

    # Use predefined opening book
    if board.fen() in opening_book:
        return chess.Move.from_uci(opening_book[board.fen()]).uci()

    # Get all legal moves
    moves = list(board.legal_moves)

    # Handle threefold repetition
    if board.can_claim_threefold_repetition():
        print("Threefold repetition detected. Filtering moves to avoid repetition.")
        moves = [move for move in moves if not leads_to_repetition(board, move)]

    if not moves:
        return None  # No legal moves available (game over or stalemate)

    # Use Stockfish for critical positions
    if len(moves) < 10 or board.is_check():
        try:
            with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
                result = engine.play(board, chess.engine.Limit(time=0.1))
                return result.move.uci()
        except Exception as e:
            print(f"Stockfish error: {e}. Falling back to heuristics.")

    # Score moves and choose the best one
    best_move = select_best_move(board)
    return best_move.uci() if best_move else random.choice(moves).uci()



5. Explore Advanced Techniques
Reinforcement Learning:
Use self-play to train your bot over time.

Create a dataset of games played by the bot.
Use frameworks like TensorFlow or PyTorch to train a neural network.
Reward positive outcomes (e.g., wins) and penalize negative ones.
Dataset Training:
Use professional chess datasets (e.g., PGN files) to train your bot.

Extract features from positions (e.g., piece positions, material balance).
Train a model to predict the best move.
Parallel Processing:
Evaluate multiple moves simultaneously to speed up decision-making.

In [ ]:
from multiprocessing import Pool

def evaluate_moves_parallel(game, moves):
    with Pool(processes=4) as pool:
        results = pool.map(score_move, [(game, move) for move in moves])
    return results


In [ ]:
from Chessnut import Game
import random


def score_move(game, move):
    """
    Score a move based on the value of the piece being captured.

    Args:
        game: The current game object.
        move: The move in UCI notation.

    Returns:
        An integer score for the move.
    """
    # Piece values
    piece_value = {"p": 1, "n": 3, "b": 3, "r": 5, "q": 9, "k": 0}
    
    # Determine the target square
    target_square = move[2:4]
    
    # Get the piece on the target square
    piece = game.board.get_piece(Game.xy2i(target_square))
    
    # Return the score based on the captured piece
    return piece_value.get(piece.lower(), 0) if piece != " " else 0


def chess_bot(obs):
    """
    Improved chess bot with predefined openings, move scoring, and fallback logic.
    """
    # Predefined openings
    openings = {
        "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1": "e2e4",  # King's Pawn Opening
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR b KQkq - 0 1": "d2d4",   # Queen's Pawn Opening
    }

    # Check if the current board state matches an opening
    if obs.board in openings:
        return openings[obs.board]  # Play the predefined opening move

    # Default move logic
    game = Game(obs.board)
    moves = list(game.get_moves())

    if not moves:
        return None  # No legal moves available (game over or stalemate)

    # Score all moves
    scored_moves = [(move, score_move(game, move)) for move in moves]

    # Choose the move with the highest score
    best_move = max(scored_moves, key=lambda x: x[1], default=(None, 0))[0]

    return best_move if best_move else random.choice(moves)  # Fallback to a random move


# Example integration for testing
if __name__ == "__main__":
    from kaggle_environments import make

    # Create the chess environment
    env = make("chess", debug=True)

    # Test your chess bot against a random agent
    result = env.run([chess_bot, "random"])

    # Display the final game state
    for agent in result[-1]:
        print(f"Status: {agent.status}, Reward: {agent.reward}")

    # Render the game
    env.render(mode="ipython", width=1000, height=1000)


In [ ]:
# Define the chess bot code
bot_code = """
from Chessnut import Game
import random


def score_move(game, move):
    \"""
    Score a move based on the value of the piece being captured.

    Args:
        game: The current game object.
        move: The move in UCI notation.

    Returns:
        An integer score for the move.
    \"""
    # Piece values
    piece_value = {"p": 1, "n": 3, "b": 3, "r": 5, "q": 9, "k": 0}
    
    # Determine the target square
    target_square = move[2:4]
    
    # Get the piece on the target square
    piece = game.board.get_piece(Game.xy2i(target_square))
    
    # Return the score based on the captured piece
    return piece_value.get(piece.lower(), 0) if piece != " " else 0


def chess_bot(obs):
    \"""
    Improved chess bot with predefined openings, move scoring, and fallback logic.
    \"""
    # Predefined openings
    openings = {
        "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1": "e2e4",  # King's Pawn Opening
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR b KQkq - 0 1": "d2d4",   # Queen's Pawn Opening
    }

    # Check if the current board state matches an opening
    if obs.board in openings:
        return openings[obs.board]  # Play the predefined opening move

    # Default move logic
    game = Game(obs.board)
    moves = list(game.get_moves())

    if not moves:
        return None  # No legal moves available (game over or stalemate)

    # Score all moves
    scored_moves = [(move, score_move(game, move)) for move in moves]

    # Choose the move with the highest score
    best_move = max(scored_moves, key=lambda x: x[1], default=(None, 0))[0]

    return best_move if best_move else random.choice(moves)  # Fallback to a random move
"""

# Write the bot code to main.py
with open("/kaggle/working/main.py", "w") as file:
    file.write(bot_code)

print("main.py has been saved to /kaggle/working/")

# Package the submission as a .tar.gz file
!tar -czvf /kaggle/working/submission.tar.gz -C /kaggle/working main.py

print("submission.tar.gz has been created for submission.")


In [ ]:
!cp /kaggle/input/stockfish/stockfish-ubuntu-x86-64-avx2 /kaggle/working/stockfish


In [ ]:
!chmod +x /kaggle/working/stockfish


In [ ]:
STOCKFISH_PATH = "/kaggle/working/stockfish"


In [ ]:
!/kaggle/working/stockfish


In [ ]:
import chess.engine
from Chessnut import Game
import random

# Step 1: Copy Stockfish binary to /kaggle/working
!cp /kaggle/input/stockfish/stockfish-ubuntu-x86-64-avx2 /kaggle/working/stockfish
!chmod +x /kaggle/working/stockfish

# Path to Stockfish binary
STOCKFISH_PATH = "/kaggle/working/stockfish"

def advanced_score_move(game, move):
    """
    Advanced move scoring based on piece value and positional evaluation.

    Args:
        game: The current game object.
        move: The move in UCI notation.

    Returns:
        An integer score for the move.
    """
    piece_value = {"p": 1, "n": 3, "b": 3, "r": 5, "q": 9, "k": 0}
    position_value = {"center": 0.5, "king_safety": 1.0, "pawn_structure": 0.5}

    # Piece value scoring
    target_square = move[2:4]
    piece = game.board.get_piece(Game.xy2i(target_square))
    score = piece_value.get(piece.lower(), 0) if piece != " " else 0

    # Add positional bonuses
    if move in ["e2e4", "d2d4"]:  # Example moves for controlling the center
        score += position_value["center"]

    if move in ["g1f3", "b1c3"]:  # Developing knights
        score += 0.3

    return score


def chess_bot(obs):
    """
    Improved chess bot using Stockfish and fallback logic.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation.
    """
    # Predefined openings
    openings = {
        "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1": "e2e4",  # King's Pawn Opening
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR b KQkq - 0 1": "d2d4",   # Queen's Pawn Opening
    }

    # Check if the current board state matches an opening
    if obs.board in openings:
        return openings[obs.board]  # Play the predefined opening move

    # Default move logic
    game = Game(obs.board)
    moves = list(game.get_moves())

    if not moves:
        return None  # No legal moves available (game over or stalemate)

    # Try Stockfish for advanced move selection
    try:
        with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
            # Convert the FEN to a chess board
            board = chess.Board(obs.board)
            result = engine.play(board, chess.engine.Limit(time=0.1))  # 0.1-second time limit
            return result.move.uci()
    except Exception as e:
        print(f"Stockfish error: {e}. Falling back to move scoring.")

    # Score all moves using advanced scoring
    scored_moves = [(move, advanced_score_move(game, move)) for move in moves]

    # Choose the move with the highest score
    best_move = max(scored_moves, key=lambda x: x[1], default=(None, 0))[0]

    return best_move if best_move else random.choice(moves)  # Fallback to a random move


# Example: Testing the bot in Kaggle environment
if __name__ == "__main__":
    from kaggle_environments import make

    # Create the chess environment
    env = make("chess", debug=True)

    # Test your chess bot against a random agent
    result = env.run([chess_bot, "random"])

    # Display the final game state
    for agent in result[-1]:
        print(f"Status: {agent.status}, Reward: {agent.reward}")

    # Render the game
    env.render(mode="ipython", width=1000, height=1000)

    # Save the bot logic to main.py for submission
    bot_code = """
from Chessnut import Game
import random
import chess.engine

# Path to Stockfish engine
STOCKFISH_PATH = "/kaggle/working/stockfish"

def chess_bot(obs):
    # Implementation same as above
    pass
"""
    with open("/kaggle/working/main.py", "w") as file:
        file.write(bot_code)

    # Package for submission
    !tar -czvf /kaggle/working/submission.tar.gz -C /kaggle/working main.py

    print("submission.tar.gz has been created for submission.")



In [ ]:
import chess
import chess.engine
import random

# Path to Stockfish binary
STOCKFISH_PATH = "/kaggle/working/stockfish"

# Step 1: Copy Stockfish binary to /kaggle/working (for Kaggle environment)
!cp /kaggle/input/stockfish/stockfish-ubuntu-x86-64-avx2 /kaggle/working/stockfish
!chmod +x /kaggle/working/stockfish

# Expanded opening book
opening_book = {
    "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1": "e2e4",  # King's Pawn
    "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR b KQkq - 0 1": "d2d4",    # Queen's Pawn
    "rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1": "c7c5",  # Sicilian Defense
    "rnbqkbnr/pppppppp/8/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 1": "e7e5", # Italian Game
}

# Lightweight heuristic evaluation
def evaluate_board(board):
    """
    Improved heuristic to evaluate the board position.
    """
    piece_values = {
        chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3,
        chess.ROOK: 5, chess.QUEEN: 9, chess.KING: 0
    }

    # Evaluate material balance
    score = sum(
        len(board.pieces(piece, chess.WHITE)) * value -
        len(board.pieces(piece, chess.BLACK)) * value
        for piece, value in piece_values.items()
    )

    # Positional bonuses
    score += 0.1 * len(list(board.legal_moves))  # Mobility
    if board.is_check():
        score -= 0.5  # Penalize being in check

    return score

# Minimax with depth-limited search
def minimax(board, depth, maximizing_player):
    """
    Minimax search with depth limit.
    """
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    if maximizing_player:
        max_eval = float('-inf')
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, False)
            board.pop()
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, True)
            board.pop()
            min_eval = min(min_eval, eval)
        return min_eval

def choose_move_with_search(board, depth=2):
    """
    Choose the best move using depth-limited minimax search.
    """
    best_move = None
    best_value = float('-inf')

    for move in board.legal_moves:
        board.push(move)
        board_value = minimax(board, depth - 1, False)
        board.pop()
        if board_value > best_value:
            best_value = board_value
            best_move = move

    return best_move

# Chess bot logic
def chess_bot(obs):
    """
    Chess bot that combines heuristic evaluation, minimax, and Stockfish.
    """
    board = chess.Board(obs.board)

    # Use predefined openings if available
    if board.fen() in opening_book:
        return chess.Move.from_uci(opening_book[board.fen()]).uci()

    # Use Stockfish in critical scenarios
    if len(list(board.legal_moves)) < 10 or board.is_check():
        try:
            with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
                result = engine.play(board, chess.engine.Limit(time=0.1))
                return result.move.uci()
        except Exception as e:
            print(f"Stockfish error: {e}. Falling back to heuristics.")

    # Use heuristic or minimax for general positions
    move = choose_move_with_search(board, depth=2)
    return move.uci() if move else None

# Test the bot in Kaggle environment
if __name__ == "__main__":
    from kaggle_environments import make

    # Create the chess environment
    env = make("chess", debug=True)

    # Test your chess bot against a random agent
    result = env.run([chess_bot, "random"])

    # Display game results
    for agent in result[-1]:
        print(f"Status: {agent.status}, Reward: {agent.reward}")

    # Render the game
    env.render(mode="ipython", width=1000, height=1000)

    # Save the bot to main.py for submission
    bot_code = """
from chess import Board
import random

# Chess bot logic here (compressed and simplified)
"""
    with open("/kaggle/working/main.py", "w") as file:
        file.write(bot_code)

    # Compress submission
    !tar -czvf /kaggle/working/submission.tar.gz -C /kaggle/working main.py
    print("Submission package created.")


In [ ]:
import chess
import chess.engine
import random

# Step 1: Copy Stockfish binary to /kaggle/working (for Kaggle environment)
!cp /kaggle/input/stockfish/stockfish-ubuntu-x86-64-avx2 /kaggle/working/stockfish
!chmod +x /kaggle/working/stockfish

# Path to Stockfish binary
STOCKFISH_PATH = "/kaggle/working/stockfish"

# Predefined opening moves for better efficiency in early-game stages
opening_book = {
    "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1": "e2e4",  # King's Pawn
    "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR b KQkq - 0 1": "d2d4",    # Queen's Pawn
    "rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1": "c7c5",  # Sicilian Defense
    "rnbqkbnr/pppppppp/8/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 1": "e7e5" # Italian Game
}

def evaluate_board(board):
    """
    Heuristic evaluation of the board position.
    """
    piece_values = {
        chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3,
        chess.ROOK: 5, chess.QUEEN: 9, chess.KING: 0
    }

    # Material score
    score = sum(
        len(board.pieces(piece, chess.WHITE)) * value -
        len(board.pieces(piece, chess.BLACK)) * value
        for piece, value in piece_values.items()
    )

    # Positional bonuses
    if board.is_check():
        score -= 0.5  # Penalize being in check
    score += 0.1 * len(list(board.legal_moves))  # Mobility bonus

    return score

def score_moves_with_heuristics(board):
    """
    Score all legal moves using a heuristic evaluation.
    """
    scored_moves = []
    for move in board.legal_moves:
        board.push(move)
        move_score = evaluate_board(board)
        board.pop()
        scored_moves.append((move, move_score))
    return scored_moves

def select_best_move(board):
    """
    Select the best move based on heuristic evaluation.
    """
    scored_moves = score_moves_with_heuristics(board)
    if not scored_moves:
        return None
    return max(scored_moves, key=lambda x: x[1])[0]

def chess_bot(obs):
    """
    Chess bot combining predefined openings, Stockfish, and heuristics.
    """
    board = chess.Board(obs.board)

    # Use opening book if available
    if board.fen() in opening_book:
        return chess.Move.from_uci(opening_book[board.fen()]).uci()

    # Use Stockfish for critical positions
    if len(list(board.legal_moves)) < 10 or board.is_check():
        try:
            with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
                result = engine.play(board, chess.engine.Limit(time=0.1))
                return result.move.uci()
        except Exception as e:
            print(f"Stockfish error: {e}. Falling back to heuristics.")

    # Otherwise, use heuristics
    best_move = select_best_move(board)
    return best_move.uci() if best_move else None

# Test the bot in Kaggle environment
if __name__ == "__main__":
    from kaggle_environments import make

    # Create the chess environment
    env = make("chess", debug=True)

    # Test your chess bot against a random agent
    result = env.run([chess_bot, "random"])

    # Display game results
    for agent in result[-1]:
        print(f"Status: {agent.status}, Reward: {agent.reward}")

    # Render the game
    env.render(mode="ipython", width=1000, height=1000)

    # Save the bot logic to main.py for submission
    bot_code = """
from chess import Board
import random

# Chess bot logic here (compressed and simplified)
"""
    with open("/kaggle/working/main.py", "w") as file:
        file.write(bot_code)

    # Compress submission
    !tar -czvf /kaggle/working/submission.tar.gz -C /kaggle/working main.py
    print("Submission package created.")
